<h1 style="color:red">Mini projet entrepôt et fouille de données</h1>

@author: Jonathan YUE CHUN<br>
@author: Quentin LEVAVASSEUR<br>
@author: Valentin BOUCHEVREAU

<strong style="color:blue">[1] Données</strong>

TODO: createCSV from result(result format TBD)

In [1]:
import pandas
import numpy

class DataManager:
    
    def __init__(self, data, sheetName):
        xls = pandas.ExcelFile(data)
        self.df = xls.parse(sheetname=sheetName)
    
    def exportTOcsv(self, outputName, header=True):     
        df2 = self.df.fillna("")
        df2.to_csv(outputName+'.csv', encoding='utf-8', index=False, header=header)
        
    def dropColumns(self, columnsIndex):
        self.df = self.df.drop(self.df.columns[columnsIndex], axis=1)
        
    def dropRows(self, rowsIndex):
        self.df = self.df.drop(self.df[self.df.index[rowsIndex]], axis=0)
    
    def replaceColumnValue(self, columnName, regexString):   
        self.df[columnName] = self.df[columnName].str.replace(regexString, '')
        
    def columnToNumeric(self, columnName):
        self.df[columnName] = pandas.to_numeric(self.df[columnName], errors='coerce').fillna(0).astype(numpy.int64)
        
    def getDataframe(self):
        df2 = self.df.fillna("")
        return df2

<strong style="color:blue">Extraction des feuilles xlsx et l'exporter en CSV (+Nettoyage)</strong>

<ul>
  <li>Pour les données de test, suppression des colonnes de prédictions (5 colonnes...).</li>
</ul>



In [25]:
dataManager1 = DataManager("Alphaprise.xlsx", "apprentissage")
dataManager1.replaceColumnValue("PrevisionnelAnnuel", "[^0-9.e+]")
dataManager1.columnToNumeric("PrevisionnelAnnuel")
dataManager1.columnToNumeric("NbSalaries")
#dataManager1.exportTOcsv("apprentissage", header=False)

dataManager2 = DataManager("Alphaprise.xlsx", "test")
dataManager2.dropColumns([34,35,36,37,38])
#dataManager2.exportTOcsv("test", header=False)

<strong style="color:blue">[2] Travail demandé</strong><br><br>
<strong>Analyse statistique</strong>

In [ ]:
learningData = dataManager1.getDataframe()
testData = dataManager2.getDataframe()

averagePrevisionnelAnnuel = learningData['PrevisionnelAnnuel'].mean()
averageCapaciteEmprunt = learningData['CapaciteEmprunt'].mean()
averageNbSalaries = learningData['NbSalaries'].mean()
print("averagePrevisionnelAnnuel: " + str(averagePrevisionnelAnnuel) + "\t" + "averageCapaciteEmprunt: " + str(averageCapaciteEmprunt) + "\t" + "averageNbSalaries: " + str(averageNbSalaries) + "\n")

for i in range(1,31):
    temp = learningData["P"+str(i)].mean()
    print("P" + str(i) + " average: " + str(temp)+ "\n")


<p><strong style="color:red">Conclusion sur l'analyse de la moyenne sur les colonnes numériques</strong></p>
<ul>
    <li>On peut éliminer les deux colonnes <strong>P10</strong> et <strong>P17</strong> (car valeur très inférieurs aux autres colonne P) pour les futures analyses;</li>
    <li>Les colonnes <strong>P1, P3, P20</strong> se distincte. En effet, ces colonnes là possèdent des valeurs très au-dessus de tous les autres P;</li>
    <li>Des colonnes P restants, on distincte deux groupes, notamment <strong>(P2, P4, P5, P6, P12, P13, P14, P16, P21, P22)</strong> et <strong>(P7, P8, P9, P11, P15, P18, P19, P23, P24, P25, P26, P27, P28, P29, P30)</strong></li>
    <li>Pour la moyenne sur la colonne NbSalaries, il est nécessaire de le faire en plusieurs groupes différents. C'est-à-dire: une pour les entreprises à taille humaine (entre 4 - 20 employés), une pour les PME (entre 21 à 100 employés) et une pour les grande entreprises (100+ employés)</li>
<ul>

<strong>Faire émerger des secteurs d'activité à partir des données(le nombre de secteurs d'activité identifié par la direction des ventes est de 8 secteurs, en comptant les Particuliers et les Divers, mais ce chiffre n'est en aucun cas une valeur parfaite). <br>
    <p style="color:blue">Variables à exclure</p>au moins Secteur1, Secteur2, SecteurParticulier, SecteurDivers</strong>
    
D'après le raisonnement réalisé précédemment, on exclura également <strong>P10</strong> et <strong>P17</strong>

In [ ]:
#Suppression des colonnes par rapport à l'index de la colonne (à exécuter une seule fois, sinon refaire l'importation des données)
dataManager1.dropColumns([1,2,3,4, 20, 27])

<strong>Eclatement de la donnée source en prenant compte l'hypothèse énoncé plus haut sur la colonne NbSalaries, c'est-à-dire qu'on cherchera à regrouper les différents catégories de clients ensembles...
Dans la suite, on cherchera à faire la même chose en considérant sur les colonnes P{1..30}</strong>
<ul>
    <li>données des entreprises où le nombre de salariés est inférieur à 5</li>
    <li>données des entreprises (PME): 21 à 100 employés</li>
    <li>données des entreprises de tailles importantes</li>
</ul>

In [65]:
smallCompanyData = dataManager1.getDataframe().loc[dataManager1.getDataframe()['NbSalaries'] < 6]
mediumCompanyData = dataManager1.getDataframe().loc[(dataManager1.getDataframe()['NbSalaries'] > 6) & (dataManager1.getDataframe()['NbSalaries'] < 100)]
bigCompanyData = dataManager1.getDataframe().loc[dataManager1.getDataframe()['NbSalaries'] > 100]

averageSmallCompanyData = smallCompanyData['NbSalaries'].mean()
averageMediumCompanyData = mediumCompanyData['NbSalaries'].mean()
averageBigCompanyData = bigCompanyData['NbSalaries'].mean()

print("Moyennes du nombres de salariés")
print("Petite entreprise/client individuel: " + str(int(averageSmallCompanyData)))
print("PME: " + str(int(averageMediumCompanyData)))
print("Grande entreprise: " + str(int(averageBigCompanyData)))
print("")
print("Petite entreprise/client individuel:")
for i in range(1,31):
    if i != 10 and i != 17:
        temp = smallCompanyData["P"+str(i)].mean()
        print("P" + str(i) + " average: " + str(temp)+ "\n")
        
print("PME: ")        
for i in range(1,31):
    if i != 10 and i != 17:
        temp = mediumCompanyData["P"+str(i)].mean()
        print("P" + str(i) + " average: " + str(temp)+ "\n")

print("Grande entreprise: ")
for i in range(1,31):
    if i != 10 and i != 17:
        temp = bigCompanyData["P"+str(i)].mean()
        print("P" + str(i) + " average: " + str(temp)+ "\n")

Moyennes du nombres de salariés
Petite entreprise/client individuel: 1
PME: 17
Grande entreprise: 386

Petite entreprise/client individuel:
P1 average: 3542.9983561922245

P2 average: 1186.7505892512427

P3 average: 1671.3288855023882

P4 average: 1443.981208981005

P5 average: 1754.2143543635068

P6 average: 1900.9822046124077

P7 average: 503.91761556435995

P8 average: 268.7325642588631

P9 average: 230.30288631514767

P11 average: 159.15047038504548

P12 average: 1653.8181784008932

P13 average: 1315.207180737581

P14 average: 1277.790763994716

P15 average: 90.16822107081177

P16 average: 1167.8550167631788

P18 average: 97.46876562023792

P19 average: 346.28965559280743

P20 average: 3705.8893487757955

P21 average: 778.7826160723342

P22 average: 1604.3561312607965

P23 average: 428.4394442751197

P24 average: 260.6796992786752

P25 average: 22.804958854007932

P26 average: 183.1860022350911

P27 average: 14.62048460835112

P28 average: 187.94798943411544

P29 average: 249.23961